In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
import numpy as np

In [2]:
BATCH_SIZE = 128
EPOCHS = 30
LAYER_SIZE = 30
S_LAYER_SIZE = 15
LATENT_SIZE = 3

In [3]:
# Load the data from the file

saved = np.load("testing.npy", allow_pickle=True)

# Make a trainging and testing batch
train_data = torch.Tensor(saved[:int(len(saved)*0.5)])
test_data = torch.Tensor(saved[int(len(saved)*0.5):])
print(train_data.shape)
INPUT_SIZE = len(test_data[0])
INPUT_SIZE

torch.set_printoptions(linewidth=140)

torch.Size([50000, 11])


In [4]:
device = "cuda"

In [5]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        # Encoder
        self.l1 = nn.Linear(INPUT_SIZE, S_LAYER_SIZE)
        self.d1 = nn.Linear(S_LAYER_SIZE, LAYER_SIZE)
        self.d2 = nn.Linear(LAYER_SIZE, S_LAYER_SIZE)
        self.l2a = nn.Linear(S_LAYER_SIZE, LATENT_SIZE)
        self.l2b = nn.Linear(S_LAYER_SIZE, LATENT_SIZE)
        
        # Decoder
        self.l3 = nn.Linear(LATENT_SIZE, S_LAYER_SIZE)
        self.d3 = nn.Linear(S_LAYER_SIZE, LAYER_SIZE)
        self.d4 = nn.Linear(LAYER_SIZE, S_LAYER_SIZE)
        self.l4 = nn.Linear(S_LAYER_SIZE, INPUT_SIZE)
            
    # Run some data through the encoder
    def encode(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.d1(x))
        x = F.relu(self.d2(x))
        # return the mu and the sigma
        return self.l2a(x), self.l2b(x)
    
    # The reparameterization trick, taken from https://github.com/pytorch/examples/blob/master/vae/main.py
    def reparameterize(self, mu, sigma):
        std = torch.exp(0.5*sigma)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def decode(self, x):
        x = F.relu(self.l3(x))
        x = F.relu(self.d3(x))
        x = F.relu(self.d4(x))
        return torch.sigmoid(self.l4(x)) # sigmoid vs tanh
    
    def forward(self, x):
        # TODO: check the shape of x to be sure we have the right input
        mu, sigma = self.encode(x)
        z = self.reparameterize(mu, sigma)
        # The loss function needs the mu and the sigma so just return them here
        return self.decode(z), mu, sigma

In [6]:
# Reconstruction + KL divergence losses summed over all elements and batch
# Taken from https://github.com/pytorch/examples/blob/master/vae/main.py
def loss_function(recon_x, x, mu, sigma):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + sigma - mu.pow(2) - sigma.exp())
#     return BCE + KLD
    return BCE

In [7]:
def train(e_count, model, optimizer):
    data = train_data
    model.train()
    train_loss = 0
    for i in range(0, len(data), BATCH_SIZE):
        batch = data[i:i+BATCH_SIZE].to(device)
        model.zero_grad()
        recons, mu, sigma = model(batch)
        loss = loss_function(recons, batch, mu, sigma)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
    print(f"Epoch: {e_count}, Loss: {train_loss/len(data)}")        
        
    
    

In [8]:
# Lets use CPU
def test(e_count, model, optimizer):
    model.eval()
    test_loss = 0
    for i in range(0, len(test_data), BATCH_SIZE):
        batch = test_data[i:i+BATCH_SIZE].to(device)
        recons, mu, sigma = model(batch)
        loss = loss_function(recons, batch, mu, sigma)
        test_loss += loss.item()
    print(f"TEST Epoch: {e_count}, Loss: {test_loss/len(test_data)}") 
        
        

In [9]:
train_data=train_data[torch.randperm(train_data.size()[0])]
test_data=test_data[torch.randperm(test_data.size()[0])]

In [10]:
vae1 = VAE().to(device)
opt1 = optim.Adam(vae1.parameters(), lr=0.001)

In [11]:
for i in range(EPOCHS):
    train(i, vae1, opt1)

Epoch: 0, Loss: 6.691904862060547
Epoch: 1, Loss: 6.273809728393554
Epoch: 2, Loss: 6.265792756347656
Epoch: 3, Loss: 6.235237529296875
Epoch: 4, Loss: 6.233139678955078
Epoch: 5, Loss: 6.232189509887696
Epoch: 6, Loss: 6.231390802612305
Epoch: 7, Loss: 6.23043057006836
Epoch: 8, Loss: 6.228728135986328
Epoch: 9, Loss: 6.223816053466797
Epoch: 10, Loss: 6.211446729125977
Epoch: 11, Loss: 6.204728806152343
Epoch: 12, Loss: 6.2031981982421875
Epoch: 13, Loss: 6.202870626220703
Epoch: 14, Loss: 6.202645319824219
Epoch: 15, Loss: 6.202445769042969
Epoch: 16, Loss: 6.202285954589843
Epoch: 17, Loss: 6.2021555773925785
Epoch: 18, Loss: 6.20204214477539
Epoch: 19, Loss: 6.2019484643554685
Epoch: 20, Loss: 6.201865169677735
Epoch: 21, Loss: 6.201796927490235
Epoch: 22, Loss: 6.20172682800293
Epoch: 23, Loss: 6.20166466796875
Epoch: 24, Loss: 6.201608460693359
Epoch: 25, Loss: 6.201556454467774
Epoch: 26, Loss: 6.2015172229003905
Epoch: 27, Loss: 6.201481577148438
Epoch: 28, Loss: 6.20144761535

In [12]:
import gym
env = gym.make("InvertedPendulum-v2")
env.reset()
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0] 
action_low = env.action_space.low[0]
action_high = env.action_space.high[0]
state_low = -10.0
state_high = 10.0

In [13]:
def descale(x):
    # State
    ((x[:, 0].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 1].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 2].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 3].mul_(state_high-state_low)).add_(state_low)).to(device)

    # Action
    ((x[:, 4].mul_(action_high-action_low)).add_(action_low)).to(device)
    
    # State
    ((x[:, 5].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 6].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 7].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 8].mul_(state_high-state_low)).add_(state_low)).to(device)
    
    # Reward
    (x[:, 9].mul_(20.0)).to(device)
    
    # Done
    (x[:, 10].round_()).to(device)
    
    return x

In [14]:
from torch.autograd import Variable

print("Action")
for i in range(10):
    sample = torch.FloatTensor(2, 11).uniform_(0, 1).to("cuda")
    print(sample[0][5])
    x = vae1(sample)
    print(x[0][0][5], '\n')

Action
tensor(0.3676, device='cuda:0')
tensor(0.5249, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.6526, device='cuda:0')
tensor(0.4958, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.4088, device='cuda:0')
tensor(0.5066, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.8574, device='cuda:0')
tensor(0.5033, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.4380, device='cuda:0')
tensor(0.5060, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.9765, device='cuda:0')
tensor(0.5776, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1388, device='cuda:0')
tensor(0.4936, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.4269, device='cuda:0')
tensor(0.5162, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1585, device='cuda:0')
tensor(0.5085, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.6349, device='cuda:0')
tensor(0.4967, device='cuda:0', grad_fn=<SelectBackward>) 



In [15]:
print("Reward")
for i in range(10):
    sample = torch.FloatTensor(2, 11).uniform_(0, 0.15).to("cuda")
    print(sample[0][9])
    x = vae1(sample)
    print(x[0][0][9], '\n')

Reward
tensor(0.0650, device='cuda:0')
tensor(0.0851, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1283, device='cuda:0')
tensor(0.0853, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0859, device='cuda:0')
tensor(0.0862, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1083, device='cuda:0')
tensor(0.0832, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0277, device='cuda:0')
tensor(0.0800, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0565, device='cuda:0')
tensor(0.0813, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0388, device='cuda:0')
tensor(0.0852, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1059, device='cuda:0')
tensor(0.0902, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1373, device='cuda:0')
tensor(0.0913, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0393, device='cuda:0')
tensor(0.0831, device='cuda:0', grad_fn=<SelectBackward>) 



In [18]:
torch.set_printoptions(precision=3, sci_mode=False, linewidth=140, profile=None)
import random
for i in range(5):
    k = random.randint(1, 400)
    sample = test_data[k].to(device)
    x, m, s = vae1(sample)
    loss = loss_function(x, sample, m, s)
    
    x = x.to("cpu")
    sample = sample.to("cpu")
    
    print(f"Original:       {sample}")
    sample = sample.to(device)
    print(f"Reconstruction: {x}\n")
    print(f"Latent: {list(vae1.encode(sample))[0]}")
    print(f"\nLoss: {loss}\n\n")
    

Original:       tensor([0.500, 0.500, 0.500, 0.500, 0.347, 0.500, 0.501, 0.484, 0.540, 0.050, 0.000])
Reconstruction: tensor([    0.501,     0.497,     0.501,     0.499,     0.347,     0.498,     0.499,     0.487,     0.531,     0.049,     0.000],
       grad_fn=<CopyBackwards>)

Latent: tensor([-1.519,  5.582, 12.349], device='cuda:0', grad_fn=<AddBackward0>)

Loss: 6.3860697746276855


Original:       tensor([0.505, 0.492, 0.579, 0.336, 0.761, 0.509, 0.484, 0.605, 0.275, 0.050, 1.000])
Reconstruction: tensor([0.503, 0.493, 0.576, 0.335, 0.778, 0.507, 0.484, 0.597, 0.279, 0.048, 1.000], grad_fn=<CopyBackwards>)

Latent: tensor([8.104, 3.041, 5.381], device='cuda:0', grad_fn=<AddBackward0>)

Loss: 6.0994720458984375


Original:       tensor([0.498, 0.504, 0.460, 0.596, 0.710, 0.497, 0.507, 0.483, 0.542, 0.050, 0.000])
Reconstruction: tensor([    0.498,     0.501,     0.460,     0.593,     0.731,     0.496,     0.503,     0.480,     0.544,     0.049,     0.000],
       grad_fn=<CopyBack

In [17]:
sample = Variable(torch.rand(1, 3))